In [55]:
%matplotlib ipympl
import sys

import hydra
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torchaudio
import lightning.pytorch as pl
from scipy.stats import gaussian_kde
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import numpy as np


from IPython.display import Audio, display

from pathlib import Path
from egaznepy.visualize import apply_plot_style
from src.data.mocha_timit_datamodule import MTIMITDataModule
from src.models.pitch_module import PitchModule
from src.models.components.metrics import RCAMetric


apply_plot_style(0.9)
plt.rcParams['text.usetex'] = False

root_dir = Path(globals()['_dh'][0]).parent
print(root_dir)
import os
os.chdir(root_dir)

#config_path_predict = 'logs/debug/runs/2024-05-29_17-03-54/.hydra/config.yaml'
config_path_predict = 'weights/joint/2024-05-29_17-03-54/.hydra/config.yaml'
#ckpt_path = 'logs/debug/runs/2024-05-29_17-03-54/checkpoints/epoch_135.ckpt'
ckpt_path = 'weights/joint/2024-05-29_17-03-54/checkpoints/epoch_135.ckpt'

#config_path_eval = 'logs/eval/runs/2024-06-02_17-08-12/.hydra/config.yaml'
config_path_eval = 'logs/eval/runs/2024-07-01_13-23-14/.hydra/config.yaml'

/dataHDD/chsaikeerthi/2024-chirumamilla


In [56]:
config_predict = OmegaConf.load(config_path_predict)
datamodule = hydra.utils.instantiate(config_predict.data, slice_length=10)
pitch_module: PitchModule = hydra.utils.instantiate(config_predict.model)

config_eval = OmegaConf.load(config_path_eval)
datamodule = hydra.utils.instantiate(config_eval.data, slice_length=10)
amazon_module: PitchModule = hydra.utils.instantiate(config_eval.model)
del config_predict.trainer.default_root_dir
del config_eval.trainer.default_root_dir
config_predict.trainer.limit_predict_batches = 1

trainer = hydra.utils.instantiate(config_predict.trainer) 
arr_predict = trainer.predict(model=pitch_module, datamodule=datamodule, ckpt_path=ckpt_path)
arr_eval = trainer.predict(model=amazon_module, datamodule=datamodule)

/dataHDD/chsaikeerthi/2024-chirumamilla/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'extractor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['extractor'])`.
/dataHDD/chsaikeerthi/2024-chirumamilla/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.


Preparing data...
Data directory: data/
Should implement the code to check if data is already downloaded and extracted.for TIMIT
Data prepared successfully.
Data folder: data/TIMIT


Restoring states from the checkpoint path at weights/joint/2024-05-29_17-03-54/checkpoints/epoch_135.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at weights/joint/2024-05-29_17-03-54/checkpoints/epoch_135.ckpt


Train set length: 5039
Validation set length: 630
Test set length: 629
Data set-up done successfully.


/dataHDD/chsaikeerthi/2024-chirumamilla/myenv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


...

/dataHDD/chsaikeerthi/2024-chirumamilla/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'encoder' removed from hparams because it cannot be pickled


Preparing data...
Data directory: data/
Should implement the code to check if data is already downloaded and extracted.for TIMIT
Data prepared successfully.
Data folder: data/TIMIT


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Data set-up done successfully.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [61]:
# Extract the first 4 elements - our model
pitches_predict = arr_predict[0][0:4] # these are the values predicted from our model
pitches_eval = arr_eval[0][0:4] # these are the values from the opus model
# gt

corresponding_data = next(iter(datamodule.test_dataloader())) # this is coming from the data module, it has speech pitch from the consensustruth and probability from the consensus truth 
corresponding_audio = corresponding_data[0][0:4] # this is the audio or the .wav file
pitches_gt = corresponding_data[1][0:4] # this is the pitch from the consensus truth
probability_gt = corresponding_data[2][0:4] # this is the probability from the consensus truth
pitches_gt = pitch_module.decoder.idx_pitch[pitches_gt.argmax(dim=-1)]


In [62]:
true_count = torch.eq(pitches_eval, pitches_gt).sum().item()
false_count = torch.eq(pitches_eval, pitches_gt).numel() - true_count

print(f"Number of Trues: {true_count}")
print(f"Number of Falses: {false_count}")
accuracy = true_count / (true_count + false_count)
print(f"Accuracy: {accuracy:.4f}")

Number of Trues: 2902
Number of Falses: 1098
Accuracy: 0.7255


In [63]:
rca = RCAMetric()
print('Ours', rca(pitches_predict, pitches_gt,probability_gt))

Ours tensor(0.7612)


In [64]:
rca = RCAMetric()
print('Amazon', rca(pitches_eval, pitches_gt,probability_gt))

Amazon tensor(0.4070)
